In [3]:
from llm_utils import *

In [4]:
lm = OAI_LM(port=8008)

2025-05-13 18:53:32.845 | INFO     | llm_utils.lm.oai_lm:__init__:81 - Using default model: openai/Qwen/Qwen3-32B-AWQ


In [ ]:
from pydantic import BaseModel


class Output(BaseModel):
    answer: str


lm("what is your name?", guid)

AssertionError: response_format must be a pydantic model, <class 'str'> provided

In [91]:
from openai import OpenAI
from pydantic import BaseModel

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8008/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
model = models.data[0].id


class People(BaseModel):
    name: str
    age: float
    is_student: bool
    hobbies: list[str]


import re
from pydantic import BaseModel
from typing import Type


import re
from pydantic import BaseModel
from typing import Type


import re
from pydantic import BaseModel
from typing import Type


def guided_regex_generator(
    model_class: Type[BaseModel], num_think_token: int = 50
) -> str:
    """
    Build a regex that enforces:
      - A <think>...</think> block of up to num_think_token*4 chars, THEN directly the JSON
        OR
      - A <think>...</think> block longer than that, THEN the stop-thinking instruction,
        THEN the JSON.

    Args:
      model_class: your Pydantic model defining the JSON schema.
      num_think_token: approximate token budget for <think> (1 token ≈ 4 chars).

    Returns:
      A single regex pattern string (use with re.DOTALL).
    """
    max_chars = num_think_token * 4

    # Build a simple JSON‐field regex from your Pydantic model
    parts = []
    for name, field in model_class.model_fields.items():
        t = field.annotation
        if t is str:
            val = r'"[^"]*"'
        elif t is int:
            val = r"\d+"
        elif t is float:
            val = r"\d+(?:\.\d+)?"
        elif t is bool:
            val = r"(?:true|false)"
        else:
            val = r".+?"
        parts.append(rf'"{name}"\s*:\s*{val}')
    json_pattern = r"\{\s*" + r"\s*,\s*".join(parts) + r"\s*\}"

    # The stop-thinking instruction to inject when over budget

    # Branch A: short think, no stop-instruction
    short_branch = rf"<think>\n" rf".{{0,{max_chars}}}" rf"</think>\n" + json_pattern

    # Branch B: long think, require stop-instruction
    long_branch = (
        rf"<think>\n"
        # at least max_chars+1 chars, non-greedy so we don't swallow the stop tag
        rf".{{0,{max_chars}}}"
        r"\n...\nConsidering the limited time by the user, I have to give the solution based on the thinking directly now\.\n"
        r"</think>\.\n\n" + json_pattern
    )

    # Combine with alternation, and DOTALL when compiling so . matches newlines
    full_regex = rf"^(?:{short_branch}|{long_branch})$"
    return long_branch  # short_branch


json_schema = People.model_json_schema()

prompt = "Generate a JSON with the name and age of one random person."
completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": prompt + "/no_think",
        }
    ],
    # extra_body={"guided_regex": guided_regex_generator(People, num_think_token=1000)},
    extra_body={"guided_json": json_schema},
    # max_tokens,
)

In [93]:
print("reasoning_content: ", completion.choices[0].message.reasoning_content)

print(completion.choices[0].message.content)

reasoning_content:  


{"name": "Emily Johnson", "age": 28, "is_student": false, "hobbies": ["reading", "hiking"]}


In [ ]:


client = OpenAI(base_url="http://0.0.0.0:8008/v1", api_key="dummy")
completion = client.beta.chat.completions.parse(
    model="Qwen/Qwen3-32B-AWQ",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "My name is Cameron, I'm 28. What's my name and age?/no_think"},
    ],
    response_format=Info,
    extra_body=dict(guided_decoding_backend="outlines"),
)

message = completion.choices[0].message
print(message)
assert message.parsed
print("Name:", message.parsed.name)
print("Age:", message.parsed.age)

ParsedChatCompletionMessage[Info](content='{"name": "Cameron", "age": 28}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, parsed=Info(name='Cameron', age=28), reasoning_content='\n\n')
Name: Cameron
Age: 28


In [100]:
print(
completion.choices[0].message.reasoning_content)

In [ ]:
from pydantic import BaseModel
from openai import OpenAI


class Info(BaseModel):
    name: str
    age: int


lm = OAI_LM(port=8008, cache=False)
lm("what is your name?/think", response_format=Info)

2025-05-13 18:56:27.293 | INFO     | llm_utils.lm.oai_lm:__init__:81 - Using default model: openai/Qwen/Qwen3-32B-AWQ


Info(name='Qwen', age=3)

"\nOkay, the user is asking for my name. I need to provide my name first. I should also add a brief introduction to help the user understand my identity and capabilities. This way, the user knows they're interacting with Qwen, a large language model developed by Alibaba Cloud. I should keep the response friendly and open-ended to encourage further interaction. Let me make sure the response is concise and clear.\n"